In [53]:
def lammpstrj(NWaterMolecule,Natoms_ads,Natoms_surface,input_lammps_name,file_path):
    Natoms = 3 * NWaterMolecule + Natoms_ads + Natoms_surface
    def extract_masses(file_path):
        start_keyword = "Masses"
        end_keyword = "Atoms"

        with open(file_path, 'r') as file:
            lines = file.readlines()

        start_index, end_index = None, None
        for i, line in enumerate(lines):
            if start_keyword in line:
                start_index = i + 1
            elif end_keyword in line and start_index is not None:
                end_index = i  
                break


        if start_index is not None and end_index is not None:
            extracted_data = [line.strip() for line in lines[start_index:end_index] if line.strip()]
            return extracted_data
        else:
            return None



    masses_data = extract_masses(file_path)
    if masses_data:

        data = []
        for line in masses_data:
            parts = line.split()
            if len(parts) >= 2:
                atom_type = parts[0]
                mass = parts[1]
                description = " ".join(parts[2:]) if len(parts) > 2 else ""
                data.append({"Atom Type": atom_type, "Mass": mass, "Description": description})

        df = pd.DataFrame(data)
        
    else:
        print("No data found between 'Masses' and 'Atoms'.")


    atoms = []
    for i in range(len(df['Description'])):
        atoms.append(df['Description'][i].split("#")[1])


    id, x, y, z = [], [], [], []
    Length = []
    with open(input_lammps_name) as f:
        for line in f:
            p = line.split()
            if len(p) == 5:
                id.append(p[1])
                x.append(np.float32(p[2]))
                y.append(np.float32(p[3]))
                z.append(np.float32(p[4]))
            if len(p) == 2:
                if p[0] != 'ITEM:':
                    Length.append(float(p[1]))

    lx, ly, lz = [], [], []
    for i in range(0,len(Length),3):
        lx.append(Length[i])
        ly.append(Length[i+1])
        lz.append(Length[i+2])

    if (lx.count(lx[0]) == len(lx)) == True:
        Lx = lx[0]
    if (ly.count(ly[0]) == len(ly)) == True:
        Ly = ly[0]
    if (lz.count(lz[0]) == len(lz)) == True:
        Lz = lz[0]


    

    
    atoms = []
    for i in range(len(df['Description'])):
        atoms.append(df['Description'][i].split("#")[1])

    ATOMS = []
    for i in range(len(atoms)):
        if atoms[i] == 'Ow':
            ATOMS.append('OW')
        elif atoms[i] == 'Hw':
            ATOMS.append('HW')
        else:
            ATOMS.append(atoms[i])

    ID = []
    for i in range(len(id)):
        ID.append(atoms[int(id[i])-1])

    
    Latice_constant = [Lx,Ly,Lz]
    Latice_matrix = [[Lx,0,0],[0,Ly,0],[0,0,Lz]]
    ID_vasp = ID[0:Natoms]

    for i in range(int(len(ID)/Natoms)):
        IDD, X, Y, Z = ID[i*Natoms:i*Natoms+Natoms],x[i*Natoms:i*Natoms+Natoms],y[i*Natoms:i*Natoms+Natoms],z[i*Natoms:i*Natoms+Natoms]
        filename = f"frame{i+1}.xyz"
        with open(filename, 'w') as file:
            file.write(str(Natoms)+ '\n')
            file.write('system at Frame = ' + str(i+1) + '\n')
            for j in range(len(IDD)):
                file.write(f"{IDD[j]:<5}{np.round(X[j]*Latice_constant[0],3):<7}{np.round(Y[j]*Latice_constant[1],3):<7}{np.round(Z[j]*Latice_constant[2],3):<7}\n")

    counts = Counter(ID_vasp)
    symbols, Ncounted = [], []
    for element, count in counts.items():
        symbols.append(element)
        Ncounted.append(count)

    water_count = 0
    for i in range(len(symbols)):
        if symbols[i] == 'Ow':
            water_count += Ncounted[i]
        elif symbols[i] == 'Hw':
            water_count += Ncounted[i]
        else:
            water_count += 0
    water_count = int(water_count/3)


    total_molecules = NWaterMolecule + 2
    SYM = []
    counted_sym = []
    for i in range(total_molecules):
        if i < len(symbols) - 2:
            SYM.append(symbols[i])
            counted_sym.append(Ncounted[i])
        else:
            SYM.append('O')
            counted_sym.append(1)
            SYM.append('H')
            counted_sym.append(2)
    res = ' '.join(SYM)
    idx = [str(d) for d in counted_sym]
    idx = ' '.join(idx)

    space= ""

    for i in range(int(len(ID)/Natoms)):
        IDD, X, Y, Z = ID[i*Natoms:i*Natoms+Natoms],x[i*Natoms:i*Natoms+Natoms],y[i*Natoms:i*Natoms+Natoms],z[i*Natoms:i*Natoms+Natoms]
        filename = f"frame{i+1}.vasp"
        with open(filename, 'w') as file:
        
            file.write('system at Frame = ' + str(i+1) + '\n')
            file.write("      "+'1.00000'+ '\n')
            for g in range(len(Latice_matrix)):
                file.write(f"{space:<6}{Latice_matrix[g][0]:<14.5f}{Latice_matrix[g][1]:<14.5f}{Latice_matrix[g][2]:<14.5f}\n")
            file.write("   "+res + '\n')
            file.write("   "+idx+ '\n')
            file.write("Direct"+ '\n')
            for j in range(len(IDD)):
                file.write(f"{space:<6}{X[j]:<14.5f}{Y[j]:<14.5f}{Z[j]:<14.5f}\n")
    
    return



In [54]:
import pandas as pd
import numpy as np
from collections import Counter
NWaterMolecule = 40
Natoms_ads = 14
Natoms_surface = 64

file_path = '/Users/alirezahosseini/Desktop/Python codes/Generating LAMMPS input files/Extracting Trajectories/data.sim'  # Replace with your file path
input_lammps_name = 'dump.sim_prod.lammpstrj'
lammpstrj(NWaterMolecule,Natoms_ads,Natoms_surface,input_lammps_name,file_path)